# Imports

In [1]:
import csv
import mysql.connector
from mysql.connector import errorcode

## Store the fetched data in a csv file

Create a method the takes the pulled data from the Azure database and adds it to a CSV file, so that it can easily be used in the Python project.

In [2]:
def write_to_csv(headers, rows):
    with open('data.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        writer.writerow(rows)

## Connect to the Azure Database for MySQL - Flexible Server

#### Obtain connection string information from the portal

In [2]:
config = {
    'host': 'ba-db.mysql.database.azure.com',
    'user': 'ba_user',
    'password': 'Pass1234',
    'database': 'data-extractor'
}

#### Construct connection string

This method is provided by the Azure documentation.

In [ ]:
try:
    conn = mysql.connector.connect(**config)
    print("Connection established")
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with the user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    cursor = conn.cursor()

    # Read data
    cursor.execute(
        "SELECT latitude, longitude, pollution, traffic_level, wind_speed, pressure, temp FROM areas_of_interest")
    rows = cursor.fetchall()
    print("Read", cursor.rowcount, "row(s) of data.")

    # Extract column headers from the cursor
    headers = [i[0] for i in cursor.description]

    # Process and write data to CSV
    write_to_csv(headers, rows)

    # Print all rows
    for row in rows:
        print("Data row = (%s, %s, %s)" % (str(row[0]), str(row[1]), str(row[2])))

    # Cleanup
    conn.commit()
    cursor.close()
    conn.close()
    print("Done.")